In [1]:
import pandas as pd 
import numpy as np
import glob

In [2]:
data = pd.read_excel("24_06_20_Speeded_naming_results.xlsx")

In [3]:
df = data[['ID', "Order","Stimulus", "STD_RESP1"]]
df = df.drop([24354,24355,24356], axis=0)
df["ID"]=df.ID.astype('int64')
df["Order"]=df.Order.astype('int64')
df.tail()

,ID,Order,Stimulus,STD_RESP1
24349,183,69,wand,wand
24350,183,218,whale,dolphin
24351,183,266,whip,whip
24352,183,269,whistle,whistle
24353,183,122,zebra,zebra


In [14]:
df['STD_RESP1'] = df['STD_RESP1'].str.strip()
df['Stimulus'] = df['Stimulus'].str.strip()

In [15]:
IDs = [x for x in list(range(101, 184)) if x != 120] #list with all subject IDs. 120 was excluded 

In [16]:
def sort_participant(num):
    participant = df[df["ID"] == num] #stores the data of each person 
    return participant.sort_values(by=['Order'])

In [29]:
def Perseverations(num):
    #### Col 1-2: Real perseverations in relation to STIMULUS
    part = sort_participant(num)
    stimuli = pd.Series(part.Order.values,index=part.Stimulus).to_dict() 
    # dictionary where each key=stimulus, value=its' order of presentation
    part.set_index('Order', inplace=True) # now order of presentation is index
    
    pers_stim = []
    pers_resp = []
    col1 = []
    
    pers_stim.append(part.Stimulus[1])
    pers_resp.append(part.STD_RESP1[1])
    col1.append(0)
    for stim, resp in zip(part.Stimulus[1:], part.STD_RESP1[1:]):
    #print(stim, resp)
        if resp not in pers_stim:
            col1.append(0)
        if resp in pers_stim:
            col1.append(1)
        pers_stim.append(stim)
        
    part["real_Stim"] = col1
    
    Lag1 = []
    part = part.reset_index()
    for ind in part.index:
        if part['real_Stim'][ind] == 0:
            Lag1.append(np.NaN)
        else:
            if part['STD_RESP1'][ind] in stimuli:
                diff = part['Order'][ind] - stimuli[part['STD_RESP1'][ind]]
                Lag1.append(diff)
                
    part["Lag1"] = Lag1
    
    ###Col 3-4: Real perseverations in relation to RESPONSE
    prev_resp = []
    col3 = []
    prev_resp.append(part.STD_RESP1[0])
    col3.append(0)
    
    for resp in part.STD_RESP1[1:]:
        #print(stim, resp)
        if resp not in prev_resp:
            col3.append(0)
        #if resp in prev_stim:
        else:
            col3.append(1)
        prev_resp.append(resp)

    part["real_Resp"] = col3
    part["Lag2"] = part.groupby(['STD_RESP1'])['Order'].diff() #.fillna("NA")
    
    rev_part = part[::-1]
    pers_stimf = []
    pers_respf = []
    col5 = []
    
    pers_stimf.append(rev_part.Stimulus[len(rev_part.Stimulus)-1])
    pers_respf.append(rev_part.STD_RESP1[len(rev_part.STD_RESP1)-1])
    col5.append(0)
    
    for stim, resp in zip(rev_part.Stimulus[1:], rev_part.STD_RESP1[1:]):
        #print(stim, resp)
        if resp not in pers_stimf:
            col5.append(0)
        if resp in pers_stimf:
            col5.append(1)
        pers_stimf.append(stim)
        
    rev_part["false_Stim"] = col5
    
    Lag3 = []
    stimuli = pd.Series(rev_part.Order.values,index=rev_part.Stimulus).to_dict()
    for ind in rev_part.index:
        if rev_part['false_Stim'][ind] == 0:
            Lag3.append(np.NaN)
        else:
            if rev_part['STD_RESP1'][ind] in stimuli:
                diff = rev_part['Order'][ind] - stimuli[rev_part['STD_RESP1'][ind]]
                Lag3.append(diff)
    
    rev_part["Lag3"] = Lag3
    
    ## cols 7-8
    
    prev_respf = []
    col6 = []
    prev_respf.append(rev_part.STD_RESP1[len(rev_part.STD_RESP1)-1])
    col6.append(0)

    for resp in rev_part.STD_RESP1[1:]:
        #print(stim, resp)
        if resp not in prev_respf:
            col6.append(0)
        #if resp in prev_stim:
        else:
            col6.append(1)
        prev_respf.append(resp)
    #df["real_Stim"] = col1
    #df["real_Resp"] = col3
    
    rev_part["false_Resp"] = col6
    
    rev_part["Lag4"] = rev_part.groupby(['STD_RESP1'])['Order'].diff()
    
    return rev_part[::-1]
    
    

In [30]:
for ID in IDs:
    part = Perseverations(ID)
    part.to_excel("test" + str(ID) + ".xlsx")

C:\Users\Omistaja\Anaconda3\lib\site-packages\ipykernel_launcher.py:72: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Users\Omistaja\Anaconda3\lib\site-packages\ipykernel_launcher.py:84: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Users\Omistaja\Anaconda3\lib\site-packages\ipykernel_launcher.py:104: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.h

In [32]:
all_data = pd.DataFrame()
for f in glob.glob("D:/EMCL_Internship/Perseverations_small_29_06/test1*.xlsx"):
    df = pd.read_excel(f)
    all_data = all_data.append(df,ignore_index=True)
    
all_data.to_excel("06_29_Perseverations_fixed_bugs.xlsx")

In [34]:
#### checking what we saved 

correct = pd.read_excel("06_29_Perseverations_fixed_bugs.xlsx") 
wrong = pd.read_excel("06_24_20_Perseverations_full.xlsx")
correct.head()

,Order,ID,Stimulus,STD_RESP1,real_Stim,Lag1,real_Resp,Lag2,false_Stim,Lag3,false_Resp,Lag4
0,1,101,sword,sword,0,NaN,0,NaN,0,NaN,0,NaN
1,2,101,microscope,NaN,0,NaN,0,NaN,0,NaN,0,NaN
2,3,101,boots,boots,0,NaN,0,NaN,0,NaN,1,-285.0
3,4,101,elephant,elephant,0,NaN,0,NaN,0,NaN,0,NaN
4,5,101,bat_(baseball),bat_(baseball),0,NaN,0,NaN,0,NaN,0,NaN


In [44]:
correct["real_Stim"].count()

24354

In [45]:
wrong["real_Stim"].count()

24354